# RAG with LangChain & Milvus: Sentence Transformers 本地模型 Demo

> 适合无 OpenAI Key、企业内网、敏感知识库等场景。

**依赖安装**
```bash
pip install langchain pymilvus sentence-transformers ollama
```
Milvus 推荐 docker 一键启动，
文档示例路径：docs/enterprise_guide.txt


In [1]:
# 0. 导入依赖 & 环境准备
import os
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, list_collections

In [2]:
# 选择本地向量模型
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# ===== 1. 加载文档并分块 =====
loader = TextLoader("docs/enterprise_guide.txt", encoding="utf-8")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=80)
docs = text_splitter.split_documents(documents)
print(f'文档共分块: {len(docs)} 个')

/var/folders/ft/rfbtvz_n5l9dg007x5lsk7jw0000gn/T/ipykernel_18736/4166844520.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


文档共分块: 5 个


In [3]:

# ===== 2. 文本向量化 =====
texts = [doc.page_content for doc in docs]
vectors = embed_model.embed_documents(texts)
print(f'已完成文本向量化，向量维度: {len(vectors[0])}')

已完成文本向量化，向量维度: 384


In [4]:
# 3. 连接 Milvus 并初始化 Collection
connections.connect(host="localhost", port="19530")
collection_name = "rag_demo_local"
dim = len(vectors[0])

if collection_name not in list_collections():
    fields = [
        FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=dim),
        FieldSchema(name="content", dtype=DataType.VARCHAR, max_length=2048),
    ]
    schema = CollectionSchema(fields)
    collection = Collection(name=collection_name, schema=schema)
    collection.create_index(
        field_name="embedding",
        index_params={"index_type": "IVF_FLAT", "metric_type": "L2", "params": {"nlist": 128}}
    )
else:
    collection = Collection(collection_name)

In [5]:
# ===== 4. 插入数据到 Milvus =====
milvus_data = [
    vectors,   # embedding
    texts,     # content
]
mr = collection.insert(milvus_data)
collection.load()
print(f'已插入 {len(texts)} 条分块')


已插入 5 条分块


---
**检索与问答部分，可根据你的 LLM 替换**

这里默认只检索并打印相关片段。如果你有自己的本地 LLM（如 Qwen/DeepSeek/Llama），
直接拼 prompt 喂给模型即可。


In [6]:
# 5. 用户输入问题并检索相关内容
user_query = input('请输入你的问题: ')
query_vec = embed_model.embed_query(user_query)
search_params = {"metric_type": "L2", "params": {"nprobe": 10}}
results = collection.search(
    data=[query_vec],
    anns_field="embedding",
    param=search_params,
    limit=4,
    output_fields=["content"]
)
retrieved_chunks = [hit.entity.get("content") for hit in results[0]]
for idx, chunk in enumerate(retrieved_chunks, 1):
    print(f'相关片段{idx}：{chunk[:120]}...')


请输入你的问题:  我和她在哪里认识的


相关片段1：懒猫带给我的，是丰富的硬件资源和社群沟通，以及售后的专业和及时。花钱买省心，剩下抄作业。大抵如此了。...
相关片段2：去年与她相识于杭州，是在西湖的游船上。参加活动过后，我想在杭州逗留一天，看看被世人称为眼泪的西湖水。没做攻略，匆匆向前台要了手册，然后来到距离最近的码头。磨磨蹭蹭之后总算开船，隔着一堆大爷大妈看着看着她在拍照，也邀请我帮她拍照。于是找她要攻...
相关片段3：image-20250608214628565
然后开账户，她的 windows 好像没有可以扫码的地方，所以我帮助她注册好，然后发给她。和她自己注册不一样的是，她的设备我登录时候我这边会弹出“安全码”，然后再发给她，这样她就就可以自己处理...
相关片段4：image-20250608214510642
实际上，她给我的文件竟然有 15 个 G 多。这种大文件如果不是自建 NAS 或者商业方案根本没有传输的办法。QQ 和微信限制大小，就连邮箱也要限速 5个 G。最早以前，我们用的办法就是，分段...


In [7]:
user_query = "我和她在哪里认识的"

context = '\n'.join(retrieved_chunks)
prompt = f'已知资料如下：\n{context}\n\n请根据以上资料回答用户问题：{user_query}'

---
你可以直接将检索片段与用户问题拼接，喂给你喜欢的本地大模型进行生成即可。例如：

```python
context = '\n'.join(retrieved_chunks)
prompt = f'已知资料如下：\n{context}\n\n请根据以上资料回答用户问题：{user_query}'
# llm = ... # 调用你的本地 LLM
# result = llm(prompt)
# print(result)
```


In [8]:
# from transformers import pipeline

# # 以 deepseek-llm-7b-chat 为例
# # llm = pipeline("text-generation", model="deepseek-ai/deepseek-llm-7b-chat", device=0)
# llm = pipeline("text-generation", model="deepseek-ai/deepseek-llm-1.3b-chat", device="mps")

# result = llm(prompt, max_new_tokens=512)[0]["generated_text"]
# print(result)

In [9]:
context = '\n'.join(retrieved_chunks)
prompt = f'已知资料如下：\n{context}\n\n请根据以上资料回答用户问题：{user_query}'

In [13]:
from ollama import Client

context = '\n'.join(retrieved_chunks)
user_query = "我和她在哪里认识的"

prompt = f"已知资料如下：\n{context}\n\n请根据以上资料回答用户问题：{user_query}"

client = Client()

response = client.chat(
    model='deepseek-r1:1.5b',  # 例如 'llama2' 或其他
    messages=[{'role': 'user', 'content': prompt}]
)

print(response['message']['content'])


<think>
嗯，我现在要解决的问题是根据提供的资料，回答“我和她哪里认识的？”这个问题。首先，我需要仔细阅读所有给定的信息，并从中找到两人之间的联系点。

首先，资料中提到“懒猫带给我的，是丰富的硬件资源和社群沟通，以及售后的专业和及时。”这说明懒猫是一个产品或服务，可能与她们的关系相关。

接下来，提到“去年与她相识于杭州，是在西湖的游船上。”这里的关键信息是，他们相遇的地方是在杭州，地点是在西湖的游船上。这是很明确的地点和时间。

然后，资料中说：“参加活动过后，我想在杭州逗留一天，看看被世人称为眼泪的西湖水。”这里“参加活动”指的是她们一起参加某项活动，之后去杭州旅游，期间去了西湖，所以她当时也在西湖活动。

接着，“没做攻略，匆匆向前台要了手册，然后来到距离最近的码头。”这说明她们可能在西湖附近有活动，赶路去了码头。码头的位置应该是杭州的一个港口或者码头，方便他们到西湖游玩。

然后，提到“磨磨蹭蹭之后总算开船，隔着一堆大爷大妈看着看着她在拍照，也邀请我帮她拍照。”这里她们一起去了码头，之后开船前往杭州的西湖边，遇到一群大爷大妈，一起拍照。这进一步确认了他们在西湖附近活动过，并且在杭州逗留游玩过西湖。

最后，“大抵如此了。”这句话可能是在总结，强调了这次相遇的情况。

综合以上信息，我可以确定两人相识的地点和时间是杭州，地点是西湖的游船上。此外，她们还一起去了杭州西湖旅游，所以这是一个很明确的认识点。
</think>

我和她于去年在杭州西湖的游船上认识。


In [14]:
retrieved_chunks

['懒猫带给我的，是丰富的硬件资源和社群沟通，以及售后的专业和及时。花钱买省心，剩下抄作业。大抵如此了。',
 '去年与她相识于杭州，是在西湖的游船上。参加活动过后，我想在杭州逗留一天，看看被世人称为眼泪的西湖水。没做攻略，匆匆向前台要了手册，然后来到距离最近的码头。磨磨蹭蹭之后总算开船，隔着一堆大爷大妈看着看着她在拍照，也邀请我帮她拍照。于是找她要攻略，一起逛三潭映月，净慈寺，讨论雷峰塔的倒下。\n\n因为苏堤，我们聊到东坡，美食以及宦海沉浮。聊到最爱的粤菜和川菜更是共同的爱好，去成都旅游的时候找她要了攻略，并且约定下次去她的城市旅游给我当导游。\n\nimage-20250608222101922\n加了微信一直零零碎碎的聊着。一个周末的早上，她找我帮忙转换视频格式。由于微信的限制无法发送大文件，于是我建议她通过懒猫网盘传给我。\n\nimage-20250608214046993\n于是开始做思想工作，把数据上传到我的家里的懒猫微服上。这里还是感谢信任和支持，没有认为我这个是一些诈骗盗取信息的网站。（毕竟曾经在学校讨论代理问题，被文科生当成黑客）',
 'image-20250608214628565\n然后开账户，她的 windows 好像没有可以扫码的地方，所以我帮助她注册好，然后发给她。和她自己注册不一样的是，她的设备我登录时候我这边会弹出“安全码”，然后再发给她，这样她就就可以自己处理登录的问题了。我给她开了懒猫相册，清单，网盘和一些好玩的 APP，除了处理这个事情之外，也希望后面也能慢慢用起来其他的功能。\n\nimage-20250608214304796\n我是一个非常不喜欢 MFA 的人，但是这个二次验证还是能够接受。常规的 MFA 是每次登录都要手动输入二次验证码，而这个相同的设备只需要一次。虽然从系统设计的角度上看二者没有太大的区别，但是还是感觉这个设计，用来节约我们浪费在二次验证上的时间。我们不是牛马，我们是人，我要相信自己的验证。\n\nimage-20250608221352408\n然后我告诉他把文件上传到网盘上，然后共享整个文件夹给我。在我的不完全测试下只有文件夹才能共享，然后操作完文件之后再做同样的操作共享给她。毕竟对比被共享人而言，这个目录是只读的，所以我们用共享给对方来完成这个操作。',
 'image-2025060821